In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [52.7 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ge

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_DE_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_multilingual_DE_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

In [4]:
from pyspark.sql.functions import *
# Read in the Review dataset as a DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         DE|      10133| RVOG49N0H1FB6|B004TACMZ8|     569741360|Bosch GMS120 Ortu...|    Home Improvement|          5|            0|          0|   N|                Y|               Super|Delivery took a l...| 2014-08-01|
|         DE|      19612| RNCMD6OLTP4HM|1846071224|     785505948|The Wheels On The...|               Bo

In [5]:
# Create the vine_table DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RVOG49N0H1FB6|          5|            0|          0|   N|                Y|
| RNCMD6OLTP4HM|          5|            1|          1|   N|                Y|
| R4AUOBI8YC0R8|          5|            0|          0|   N|                Y|
|R1VSHIJ1RHIBTE|          5|            0|          0|   N|                Y|
|R3JBLVALWSLCZD|          5|            9|         14|   N|                Y|
| RJ6GK77Y2NKCK|          4|            6|          7|   N|                Y|
| R13W0E5EDX50J|          5|            0|          1|   N|                Y|
|R2L8UN6YSPY1CK|          3|            1|          3|   N|                Y|
| RA2DFU68J832H|          4|            1|          1|   N|                Y|
|R191XKANDDV34N|          4|            0|          1|   N|     

In [6]:
vine_filtered_votes = vine_df.filter(vine_df.total_votes >= 20)
vine_filtered_votes.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3JSOBULWNE10K|          5|           14|         23|   N|                N|
| R973TDYT0B600|          4|          289|        350|   N|                Y|
|R2A78SJJS8NPUK|          5|           37|         56|   N|                Y|
| RF3JH2YYCJKRM|          4|            2|         40|   N|                Y|
|R31KTNNEZK2IDF|          2|            2|         32|   N|                Y|
|R1QTHPDQZZ4HCX|          1|            5|         86|   N|                Y|
|R2UT7AZ2X3AY1A|          4|           46|         49|   N|                Y|
| RH6ZMZVUIKMAG|          1|            5|         26|   N|                Y|
|R3TH9UIAKX1FF1|          1|            5|         22|   N|                Y|
|R3FV6QDK4DS9CI|          1|           23|         47|   N|     

In [7]:
vine_helpful = vine_filtered_votes.filter((vine_filtered_votes.helpful_votes / vine_filtered_votes.total_votes) >= 0.5)
vine_helpful.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3JSOBULWNE10K|          5|           14|         23|   N|                N|
| R973TDYT0B600|          4|          289|        350|   N|                Y|
|R2A78SJJS8NPUK|          5|           37|         56|   N|                Y|
|R2UT7AZ2X3AY1A|          4|           46|         49|   N|                Y|
|R1HAGIRT9K8YYH|          5|           19|         35|   N|                N|
|R35XPA8PFX88SH|          4|           44|         53|   N|                N|
| ROL6706VGJFC6|          5|           44|         59|   N|                N|
|R3CYZYCBJ4KXMF|          1|           15|         20|   N|                N|
|R2P7AWG36QK0LN|          5|           68|         76|   N|                Y|
|R17Q8279ZKZ630|          1|           12|         20|   N|     

In [8]:
# Filter for paid programs
paid_vine_df = vine_helpful.filter(vine_helpful.vine == "Y")
paid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R11E7I5VQ03MK9|          3|          179|        222|   Y|                N|
| RIPUS8IPBEQ3I|          4|           37|         40|   Y|                N|
|R2QREWQEZOB2CK|          5|           21|         24|   Y|                N|
| R9RWVZPQ6MZES|          5|           23|         23|   Y|                N|
|R3TWKO71IXOP02|          3|           49|         51|   Y|                N|
|R3T1ONLG4I4GVH|          5|           25|         29|   Y|                N|
|R2E1FQHAOPHG51|          5|           59|         73|   Y|                N|
| RMPKTFKVEF5VL|          4|           13|         22|   Y|                N|
|R1G2VBVXL4U7R6|          4|          121|        137|   Y|                N|
|R2D7X1L8I2JSZE|          3|           19|         21|   Y|     

In [9]:
# Filter for unpaid programs
vine_unpaid_df = vine_helpful.filter(vine_helpful.vine == "N")
vine_unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3JSOBULWNE10K|          5|           14|         23|   N|                N|
| R973TDYT0B600|          4|          289|        350|   N|                Y|
|R2A78SJJS8NPUK|          5|           37|         56|   N|                Y|
|R2UT7AZ2X3AY1A|          4|           46|         49|   N|                Y|
|R1HAGIRT9K8YYH|          5|           19|         35|   N|                N|
|R35XPA8PFX88SH|          4|           44|         53|   N|                N|
| ROL6706VGJFC6|          5|           44|         59|   N|                N|
|R3CYZYCBJ4KXMF|          1|           15|         20|   N|                N|
|R2P7AWG36QK0LN|          5|           68|         76|   N|                Y|
|R17Q8279ZKZ630|          1|           12|         20|   N|     

In [52]:
# Determine the total number of reviews, the number of 5-star reviews, 
# and the percentage of 5-star reviews for the two types of review (paid vs unpaid).
paid_five_star = paid_vine_df.filter(paid_vine_df.star_rating == 5).count()
unpaid_five_star = vine_unpaid_df.filter(vine_unpaid_df.star_rating == 5).count()
total_rev_paid = paid_vine_df.distinct().count()
total_unpaid_revs = vine_unpaid_df.distinct().count()

percent_5star_paid = paid_vine_df.filter(paid_vine_df.star_rating == 5)
percent_5star_paid = (percent_5star_paid.count() / paid_vine_df.count()) * 100

percent_5star_unpaid = vine_unpaid_df.filter(vine_unpaid_df.star_rating == 5)
percent_5star_unpaid = (percent_5star_unpaid.count() / vine_unpaid_df.count()) * 100 

print(f"The number of five star ratings for paid programs was {paid_five_star}")
print(f"The number of five star ratings for unpaid programs was {unpaid_five_star}")
print(f"The total number of paid reviews was {total_rev_paid}")
print(f"The total number of unpaid reviews was {total_unpaid_revs}")
print(f"The total percentage of 5-star reviews for paid programs was {format(percent_5star_paid, '.2f')}%")
print(f"The total percentage of 5-star reviews for unpaid programs was {format(percent_5star_unpaid, '.2f')}%") 

The number of five star ratings for paid programs was 9
The number of five star ratings for unpaid programs was 15784
The total number of paid reviews was 26
The total number of unpaid reviews was 29264
The total percentage of 5-star reviews for paid programs was 34.62%
The total percentage of 5-star reviews for unpaid programs was 53.94%
